In [1]:
import torch
import torchaudio
from seamless_communication.models.inference import Translator
import os
from pydub import AudioSegment 
import requests


In [2]:

def find_mp4_files(folder_path):
    mp4_files = []
    #print(f"Searching in folder: {folder_path}")  # Debugging line
    for dirpath, _, filenames in os.walk(folder_path):
       # print(f"Checking directory: {dirpath}")  # Debugging line
        for filename in filenames:
           # print(f"Checking file: {filename}")  # Debugging line
            if filename.lower().endswith('.mp4'):  # Case-insensitive search
                full_path = os.path.join(dirpath, filename)
               # print(f"Found mp4 file: {full_path}")  # Debugging line
                mp4_files.append(full_path)
    return mp4_files

# 1. 모든 하위 폴더 목록 찾기 (절대 경로 사용)
root_folder = '/home2/nia/nia_s2t/01. 원천데이터'
sub_folders = [f.path for f in os.scandir(root_folder) if f.is_dir()]
#print("Sub-folders:", sub_folders)

# 2. 추가 폴더가 있다고 가정하고, 각 폴더에 접근해 mp4 파일 찾기
all_mp4_files = []
for sub_folder_path in sub_folders:
   # print(f"Exploring sub-folder: {sub_folder_path}")  # Debugging line
    all_mp4_files.extend(find_mp4_files(sub_folder_path))



In [3]:
def mp4_to_wav(mp4_path):
    audio = AudioSegment.from_file(mp4_path, format="mp4")
    wav_path = mp4_path.replace(".mp4", ".wav")
    audio.export(wav_path, format="wav")
    return wav_path

In [4]:
import base64
import json
import logging
import os
import shutil
import subprocess
import tempfile
import warnings
from io import BytesIO
from pathlib import Path

import numpy as np
from ffmpy import FFmpeg, FFprobe, FFRuntimeError
with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # Ignore pydub warning if ffmpeg is not installed
    from pydub import AudioSegment


In [5]:
translator_medium = Translator("seamlessM4T_medium","vocoder_36langs", torch.device("cuda:0"))

Using the cached checkpoint of the model 'seamlessM4T_medium'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_medium'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.


In [6]:
import os
# Set ulimit
os.system('ulimit -n 4096')

# # Initialize the Translator
translator_large = Translator("seamlessM4T_large","vocoder_36langs", torch.device("cuda:0"))


Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.


In [9]:
test = all_mp4_files[:10]

### 모델 input 전처리
- mp4를 16 kHz wav 포맷으로 바꾸기

In [6]:
import numpy as np
import warnings
from pydub import AudioSegment

def convert_to_16_bit_wav(data):
    if data.dtype in [np.float64, np.float32, np.float16]:
        warnings.warn("Trying to convert audio automatically from {} to 16-bit int format.".format(data.dtype))
        data = data / np.abs(data).max()
        data = data * 32767
        data = data.astype(np.int16)
    elif data.dtype == np.int32:
        warnings.warn("Trying to convert audio automatically from {} to 16-bit int format.".format(data.dtype))
        data = data / 65538
        data = data.astype(np.int16)
    elif data.dtype == np.int16:
        pass
    else:
        raise ValueError("Audio data cannot be converted automatically from {} to 16-bit int format.".format(data.dtype))
    return data



def mp4_to_wav_16bit(mp4_path, max_length=4096):
    audio = AudioSegment.from_file(mp4_path, format="mp4")
    data = np.array(audio.get_array_of_samples())
    if audio.channels > 1:
        data = data.reshape(-1, audio.channels)
    sample_rate = audio.frame_rate
    
    data = convert_to_16_bit_wav(data)
    
    wav_path = mp4_path.replace(".mp4", "_16bit.wav")
    audio = AudioSegment(
        data.tobytes(),
        frame_rate=sample_rate,
        sample_width=data.dtype.itemsize,
        channels=(1 if len(data.shape) == 1 else data.shape[1]),
    )
    audio.export(wav_path, format="wav")

    # Additional code for resampling
    resample_rate = 16000  # Target sample rate
    waveform, sample_rate = torchaudio.load(wav_path)  # Load the existing wav file
    
    if sample_rate != resample_rate:
        resampler = torchaudio.transforms.Resample(sample_rate, resample_rate, dtype=waveform.dtype)
        waveform = resampler(waveform)
    
    # Truncate or pad audio data to fit model's max input length
    if waveform.shape[1] > max_length:
        waveform = waveform[:, :max_length]
    elif waveform.shape[1] < max_length:
        pad_amount = max_length - waveform.shape[1]
        waveform = torch.nn.functional.pad(waveform, (0, pad_amount), 'constant', 0)
    
    torchaudio.save(wav_path, waveform, resample_rate)  # Save the resampled (and possibly truncated) audio
    
    return wav_path



### 두 모델 성능 비교

In [11]:
for mp4_file in test:
    wav_file = mp4_to_wav_16bit(mp4_file)  # Convert MP4 to 16-bit WAV
    #translated_text, wav, sr = translator_large.predict(
    translated_text, wav, sr = translator_medium.predict(
        wav_file,
        "s2st",  # Change this based on your task
        "kor",  # Korean language
        ngram_filtering=True
    )
    
    print(f"Translated text: {translated_text}")

Translated text: 그렇게 해서 가는 교수의 자유성도 인정해줘야 되는 거예요. 단지 그 사호처리가 학교가 엉망으로 하고 있다는 증거내리 사항이 아니지 않느냐? 국문관은 논란이 된 직후인 지난해 삼월 논문 두 편이랑 교수 채용 기준을 단동 논문 두편으로 수정했습니다. KBS 뉴스 주현지입니다. 전남 나주의 한 농촌 마을에서
Translated text: 서울시교육청은 내일 오전 여덟시 사십분부터 전국 고등학교 일 이 삼 학년 학생 구십 오 만여 명이 올해 첫 전국 연합 학력 평가를 치른다고 밝혔습니다. 지난해는 코로나 일 구의 영향으로 학년별로 나노 평가를 실시했지만 올해는 모든 학년이 같은 날 동시에 시험을 치른다. 재택 응시자는 전국 연합학력 평가 온라인 시스템의 교시별 시작 시간에 맞춰 제공되는 문제지로 응시할 수 있지만
Translated text: 지난해 통합교육 시행 이후 장애학생들의 인지능력과 학장부진한 학생들도 많이 보이거든요. 그리고 비장애 학생들도 많이 나아지지 않을까요? 그리고 그 학교는 통합교육을 통해 통합 교육과 함께 할 수 있는 학교가 늘어나게 될 것으로 예상됩니다.
Translated text: 부산대 의원원에 입학한 뒤 이천십 년과 이천이십일 년 초 의사 국가고시 실기시험과 필기시험에 합격했습니다. 이번 입학 취소 결정으로 의사면허 취소 여부에도 영향을 끼칠 것으로 보입니다. 현행 의료법상
Translated text: 학부모들은 의무검사가 아니라 다행이라면서도 걱정이 떨어지는 것 같고 아이들이 부모가 아이를 한다는 것에 대해 좀 힘들어하고 부모다 보면 더 편안한 사실이 있을 때 아마 이렇게 뭐 억지로 한다든가 그런 부분이 너무 힘들을 것 같아요. 학교에서 실시하는 게 더 실효성이 있을 것 같아요.
Translated text: 지역 기업과 기관 사십 여곳에 취업할 기회도 어떻게 됩니다. 인재들이 디지털 영량적 코딩 관련 기술을 진입오기를 기대하고 있습니다. 학생들에게 기업체가 요구하는 인재가 될 수 있도록 지방 소멸을 받

In [12]:
for mp4_file in test:
    #print(mp4_file)
    wav_file = mp4_to_wav_16bit(mp4_file)  # Convert MP4 to 16-bit WAV
    translated_text, wav, sr = translator_large.predict(
    #translated_text, wav, sr = translator.predict(
        wav_file,
        "s2st",  # Change this based on your task
        "kor",  # Korean language
        ngram_filtering=True
    )
    
    print(f"Translated text: {translated_text}")

Translated text: 다르게 해석하는 교수의 자율성도 인정해 ⁇ 야 되는 거예요. 단지 그 사호트리가 학교가  ⁇ 망으로 하고 있다는 게 징계내리 상황이 아닌가요? 고문관은 논란이 된 직후인 지난해 3월 논문 두 편인 교수 채용 기준을 단독 논문 두편으로 수정했습니다. KBSNews 수현지입니다. 전남 나주에서
Translated text: 서울시교육청은 내일 오전 여 ⁇ 시 사십분부터 전국의 고등학교 일 이 삼 학년 학생 구십 오만여 명이 올해 첫 전국 통합학력 평가를 실시한다고 밝혔다. 지난해는 코로나 일곱의 영향으로 학년별로 나누어 평가를 실시했지만 올해는 모든 학년이 같은 날 동시에 시험을 치른다. 재택시자는 전국 연합학력 평가 온라인 시스템의 시작 시간에 맞춰 제공되는 문제로 적용할 수 있지만
Translated text: 지난해 통합교육을 실시한 이후에 모두가 함께 하는 교실이 크게 향상되고 있는 것으로 분석됐습니다. 비장애 학생들도 통합교육을 통해 장애인에 대한 부담도 줄어들었는데요.
Translated text: 부산대 의전원에 입학한 후 이천 이십 년과 이천 이백 일 년 초 의사국가고시 실기시험과 필기시험에 합격했습니다. 이번 입학 취소 결정으로 의사 면허 취소 여부에도 영향을 미칠 것으로 보입니다. 현행 의료법상
Translated text: 학부모들은 의무검사가 아니라 다행이라고 하면서도 걱정이 앞선다. 집에 와서 아이들이 부모가 아이를 낳는 것에 대해 정확성도 좀 떨어지는 것 같고 아이들이 많이 힘들고 부모보다 좀 더 편안해지는 사실이 있기 때문에 #아 아이들이 거부능력을 갖추었을 때 이런 부분이 너무 힘들 것 같아요. 학교에서 실습을 하는 것이 더 힘들 것 같습니다.
Translated text: 지역 기업과 기관 사십여 곳에서 취업할 기회도 얻게 됩니다 인재들이 디지털 영향력 코딩 관련한 기술을 지니고 오기를 기대하고 있습니다 학생들에게 기업체가 요구하는 인재가 될 수 있도록 지방소멸을 받고 지방대학을 살리기 위해
Transl

In [12]:
for mp4_file in test:
    print(mp4_file)
    wav_file = mp4_to_wav_16bit(mp4_file)  # Convert MP4 to 16-bit WAV
    translated_text, wav, sr = translator_large.predict(
    #translated_text, wav, sr = translator.predict(
        wav_file,
        "s2st",  # Change this based on your task
        "kor",  # Korean language
        ngram_filtering=True
    )
    
    print(f"Translated text: {translated_text}")

/home2/nia/nia_s2t/01. 원천데이터/24. 교육뉴스/KND_00025.mp4
Translated text: 다르게 해석하는 교수의 자율성도 인정해 ⁇ 야 되는 거예요. 단지 그 사호트리가 학교가  ⁇ 망으로 하고 있다는 게 징계내리 상황이 아닌가요? 고문관은 논란이 된 직후인 지난해 3월 논문 두 편인 교수 채용 기준을 단독 논문 두편으로 수정했습니다. KBSNews 수현지입니다. 전남 나주에서
/home2/nia/nia_s2t/01. 원천데이터/24. 교육뉴스/KND_00047.mp4
Translated text: 서울시교육청은 내일 오전 여 ⁇ 시 사십분부터 전국의 고등학교 일 이 삼 학년 학생 구십 오만여 명이 올해 첫 전국 통합학력 평가를 실시한다고 밝혔다. 지난해는 코로나 일곱의 영향으로 학년별로 나누어 평가를 실시했지만 올해는 모든 학년이 같은 날 동시에 시험을 치른다. 재택시자는 전국 연합학력 평가 온라인 시스템의 시작 시간에 맞춰 제공되는 문제로 적용할 수 있지만
/home2/nia/nia_s2t/01. 원천데이터/24. 교육뉴스/KND_00078.mp4
Translated text: 지난해 통합교육을 실시한 이후에 모두가 함께 하는 교실이 크게 향상되고 있는 것으로 분석됐습니다. 비장애 학생들도 통합교육을 통해 장애인에 대한 부담도 줄어들었는데요.
/home2/nia/nia_s2t/01. 원천데이터/24. 교육뉴스/KND_00058.mp4
Translated text: 부산대 의전원에 입학한 후 이천 이십 년과 이천 이백 일 년 초 의사국가고시 실기시험과 필기시험에 합격했습니다. 이번 입학 취소 결정으로 의사 면허 취소 여부에도 영향을 미칠 것으로 보입니다. 현행 의료법상
/home2/nia/nia_s2t/01. 원천데이터/24. 교육뉴스/KND_00020.mp4
Translated text: 학부모들은 의무검사가 아니라 다행이라고 하면서도 걱정이 앞선다. 집에 와서 아이들이 부모가 아이를 낳는 것에 대해 정확성도 좀 떨어지는 

## 실행 코드

In [15]:
from tqdm import tqdm

In [8]:
from tqdm import tqdm  # Import tqdm for progress bar
import pandas as pd  # Import pandas library
import os

# Initialize empty lists for successful conversions
video_list = []
speech_to_text_list = []

# Initialize an empty list to keep track of videos that caused errors
error_video_list = []

# Iterate through all the MP4 files
for mp4_file in tqdm(all_mp4_files):
    try:
        # Extract the base file name
        base_name = os.path.basename(mp4_file)
        
        # Convert MP4 to 16-bit WAV
        wav_file = mp4_to_wav_16bit(mp4_file)
        
        # Perform speech-to-text prediction
        translated_text, wav, sr = translator_medium.predict(
            wav_file,
            "s2st",  # Change this based on your task
            "kor",  # Korean language
            ngram_filtering=True
        )
        
        # Append to lists
        video_list.append(base_name)
        speech_to_text_list.append(translated_text)
        
    except Exception as e:
        print(f"An error occurred with video {mp4_file}: {e}")
        error_video_list.append(mp4_file)

# Create DataFrame from lists for successful conversions
df = pd.DataFrame({
    'video': video_list,
    'speech_to_text': speech_to_text_list
})

# Create DataFrame for videos that caused errors
df_errors = pd.DataFrame({
    'error_video': error_video_list
})

  3%|▎         | 175/5259 [02:05<48:16,  1.75it/s]  

An error occurred with video /home2/nia/nia_s2t/01. 원천데이터/25. 문화뉴스/KNE_00029.mp4: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-lib

 22%|██▏       | 1177/5259 [14:35<44:42,  1.52it/s]  

An error occurred with video /home2/nia/nia_s2t/01. 원천데이터/23. 국제뉴스/KNC_00080.mp4: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-lib

 25%|██▌       | 1322/5259 [16:28<39:02,  1.68it/s]

An error occurred with video /home2/nia/nia_s2t/01. 원천데이터/27. 사회일반 뉴스/KNG_00442.mp4: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-

 34%|███▎      | 1765/5259 [22:16<36:45,  1.58it/s]

An error occurred with video /home2/nia/nia_s2t/01. 원천데이터/28. 과학기술뉴스/KNH_00455.mp4: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-l

 61%|██████    | 3195/5259 [41:02<25:00,  1.38it/s]